# Amazon SageMaker Model Monitor
このノートブックは、次の方法を示しています。

* Amazon SageMaker で機械学習モデルをホストし、推論リクエスト、結果、メタデータをキャプチャします
* トレーニングデータセットを分析して、ベースライン制約を生成します
* 制約に対する違反がないかライブエンドポイントを監視する

---
## Background

Amazon SageMaker は、すべての開発者とデータサイエンティストに、機械学習モデルをすばやく構築、トレーニング、デプロイする機能を提供します。 Amazon SageMaker は、機械学習ワークフロー全体を網羅するフルマネージドサービスです。 データにラベルを付けて準備し、アルゴリズムを選択し、モデルをトレーニングしてから、展開用に調整して最適化することができます。 Amazon SageMaker を使用してモデルを本番環境にデプロイし、以前よりも予測と低コストを実現できます。

さらに、Amazon SageMaker を使用すると、デプロイするモデルの呼び出しの入力、出力、メタデータをキャプチャできます。 また、データを分析してその品質を監視することもできます。 このノートブックでは、Amazon SageMaker がこれらの機能をどのように有効にするかを学びます。

---
## Setup

開始するには、これらの前提条件が満たされていることを確認してください。

* モデルをホストする AWSリージョン を指定します。
* Amazon SageMaker に Amazon Simple Storage Service（Amazon S3） のデータへのアクセスを許可するために使用される IAMロールARN が存在します。 必要な権限を調整する方法については、ドキュメントを参照してください。
* モデルのトレーニングに使用されるデータ、追加のモデルデータ、およびモデル呼び出しからキャプチャされたデータを格納するために使用される S3バケット を作成します。 デモンストレーションの目的で、これらに同じバケットを使用しています。 実際には、それらを異なるセキュリティポリシーで分離することをお勧めします。

# 参考資料

- [公式ドキュメント](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/model-monitor.html)
- [Amazon SageMaker Model Monitor を活用したデータドリフト検知の解説](https://aws.amazon.com/jp/blogs/news/detect-data-drift-with-amazon-sagemaker-model-monitor/)

In [ ]:
%%time

# Handful of configuration

import os
import boto3
import re
import json
from sagemaker import get_execution_role, session

region = boto3.Session().region_name

role = get_execution_role()
print("RoleArn: {}".format(role))

# You can use a different bucket, but make sure the role you chose for this notebook
# has the s3:PutObject permissions. This is the bucket into which the data is captured
bucket = session.Session(boto3.Session()).default_bucket()
print("Demo Bucket: {}".format(bucket))
prefix = "sagemaker/DEMO-ModelMonitor"

data_capture_prefix = "{}/datacapture".format(prefix)
s3_capture_upload_path = "s3://{}/{}".format(bucket, data_capture_prefix)
reports_prefix = "{}/reports".format(prefix)
s3_report_path = "s3://{}/{}".format(bucket, reports_prefix)
code_prefix = "{}/code".format(prefix)
s3_code_preprocessor_uri = "s3://{}/{}/{}".format(bucket, code_prefix, "preprocessor.py")
s3_code_postprocessor_uri = "s3://{}/{}/{}".format(bucket, code_prefix, "postprocessor.py")

print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))
print("Preproc Code path: {}".format(s3_code_preprocessor_uri))
print("Postproc Code path: {}".format(s3_code_postprocessor_uri))

このノートブックの実行ロールに続行するために必要な権限があることをすばやく確認できます。 上記で指定したS3バケットに簡単なテストオブジェクトを配置します。 このコマンドが失敗した場合は、バケットに対する `s3：PutObject` 権限を持つようにロールを更新して、再試行してください。

In [ ]:
# Upload some test files
boto3.Session().resource("s3").Bucket(bucket).Object("test_upload/test.txt").upload_file(
    "test_data/upload-test-file.txt"
)
print("Success! You are all set to proceed.")

# PART A: Amazon SageMakerエンドポイントからのリアルタイム推論データの取得
エンドポイントを作成して、データキャプチャー機能の動作を紹介します。

###  学習済みのモデルをAmazon S3にアップロードする
このコードは、事前にトレーニングされたXGBoostモデルをアップロードして、すぐに展開できるようにします。このモデルは、SageMaker の XGB Churn Prediction Notebook を使用してトレーニングされています。このステップでは、独自の事前学習済みモデルを使用することもできます。すでにAmazon S3に事前学習済みのモデルがある場合は、s3_keyを指定することで代わりに追加することができます。

In [ ]:
model_file = open("model/xgb-churn-prediction-model.tar.gz", "rb")
s3_key = os.path.join(prefix, "xgb-churn-prediction-model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(s3_key).upload_fileobj(model_file)

### Amazon SageMakerにモデルをデプロイする
まずは、事前に学習された解約予測モデルの展開から始めます。ここでは、画像とモデルデータを含むモデルオブジェクトを作成します。

In [ ]:
from time import gmtime, strftime
from sagemaker.model import Model
from sagemaker.image_uris import retrieve

model_name = "DEMO-xgb-churn-pred-model-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "https://{}.s3-{}.amazonaws.com/{}/xgb-churn-prediction-model.tar.gz".format(
    bucket, region, prefix
)

image_uri = retrieve("xgboost", boto3.Session().region_name, "0.90-1")

model = Model(image_uri=image_uri, model_data=model_url, role=role)

モデルのデータ品質を監視するためにデータキャプチャーを有効にするには、`DataCaptureConfig`という新しいキャプチャーオプションを指定します。この設定では、リクエストのペイロード、レスポンスのペイロード、またはその両方をキャプチャすることができます。キャプチャーコンフィグはすべてのバリアントに適用されます。デプロイメントを実行してください。

In [ ]:
from sagemaker.model_monitor import DataCaptureConfig

endpoint_name = "DEMO-xgb-churn-pred-model-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

## デプロイされたモデルを呼び出す

これで、このエンドポイントにデータを送信して、リアルタイムで推論を得ることができます。前のステップでデータキャプチャーを有効にしたので、リクエストとレスポンスのペイロードは、いくつかの追加メタデータとともに、DataCaptureConfigで指定したAmazon Simple Storage Service（Amazon S3）のロケーションに保存されます。

このステップでは、サンプルデータを含むエンドポイントを約3分間起動します。データは指定されたサンプリング・パーセンテージに基づいてキャプチャーされ、データ・キャプチャー・オプションがオフになるまでキャプチャーが続けられます。

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
import time

predictor = Predictor(endpoint_name=endpoint_name, serializer=CSVSerializer())

# get a subset of test data for a quick test
!head -180 test_data/test-dataset-input-cols.csv > test_data/test_sample.csv
print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

with open("test_data/test_sample.csv", "r") as f:
    for row in f:
        payload = row.rstrip("\n")
        response = predictor.predict(data=payload)
        time.sleep(1)

print("Done!")

## キャプチャしたデータを見る

ここで、Amazon S3に保存されているデータキャプチャファイルをリストアップしてみましょう。呼び出しが発生した時間に基づいて整理された、異なる時間帯の異なるファイルが表示されることを期待してください。Amazon S3のパスのフォーマットは以下の通りです。

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [ ]:
s3_client = boto3.Session().client("s3")
current_endpoint_capture_prefix = "{}/{}".format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
print("Found Capture Files:")
print("\n ".join(capture_files))

次に、1つのキャプチャファイルの内容を表示します。ここでは、Amazon SageMaker固有のJSONライン形式のファイルにキャプチャされたすべてのデータが表示されます。キャプチャファイルの最初の数行を簡単に覗いてみましょう。

In [ ]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get("Body").read().decode("utf-8")


capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

最後に、1行の内容を整形されたJSONファイルで以下に示しますので、少しでもよく観察してみてください。

In [ ]:
import json

print(json.dumps(json.loads(capture_file.split("\n")[0]), indent=2))

ご覧の通り、各推論要求はjsonlファイルの1行に収められています。この行には入力と出力の両方が一緒にマージされています。この例では、ContentTypeを`text/csv`としており、`observedContentType`の値に反映されています。また、入力と出力のペイロードをキャプチャ形式でエンコードするのに使用したエンコーディングを `encoding` の値で公開しています。

要約すると、新しいパラメータを使って、エンドポイントへの入力ペイロードまたは出力ペイロードのキャプチャを有効にする方法を確認しました。また、キャプチャされたフォーマットがAmazon S3でどのように見えるかを観察しました。次に、Amazon S3で収集されたデータを監視するために、Amazon SageMakerがどのように役立つかを引き続き調べてみましょう。

# PART B: Model Monitor - ベースラインと継続的モニタリング

Amazon SageMakerは、データを収集するだけでなく、エンドポイントが観測したデータを監視・評価する機能も提供しています。これには

1. リアルタイムのトラフィックと比較するためのベースラインを作成します。
1. ベースラインの準備ができたら、継続的に評価し、ベースラインと比較するためのスケジュールを設定します。

## 1. ベースライン/トレーニングデータセットを用いた制約条件の提案

モデルをトレーニングしたトレーニングデータセットは、通常、良いベースラインデータセットです。トレーニングデータセットのデータスキーマと推論データセットのスキーマは完全に一致している必要があることに注意してください（つまり、特徴の数と順序）。

トレーニングデータセットから、Amazon SageMakerにベースラインの「制約」のセットを提案してもらい、データを探索するための記述的な「統計」を生成することができます。この例では、この例に含まれるプレトレーニングモデルをトレーニングするために使用されたトレーニングデータセットをアップロードします。すでにAmazon S3にある場合は、それを直接指定することができます。

In [ ]:
# copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = "s3://{}/{}".format(bucket, baseline_data_prefix)
baseline_results_uri = "s3://{}/{}".format(bucket, baseline_results_prefix)
print("Baseline data uri: {}".format(baseline_data_uri))
print("Baseline results uri: {}".format(baseline_results_uri))

In [ ]:
training_data_file = open("test_data/training-dataset-with-header.csv", "rb")
s3_key = os.path.join(baseline_prefix, "data", "training-dataset-with-header.csv")
boto3.Session().resource("s3").Bucket(bucket).Object(s3_key).upload_fileobj(training_data_file)

### トレーニングデータセットを使ったベースライニングジョブの作成

Amazon S3にトレーニングデータの準備ができたので、制約を `suggest` するジョブを開始します。`DefaultModelMonitor.suggest_baseline(...)`は、制約を生成するためにAmazon SageMakerが提供するModel Monitorコンテナを使用して、`ProcessingJob`を開始します。

In [ ]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri + "/training-dataset-with-header.csv",
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True,
)

### 生成された制約条件や統計情報を調べる

- statiistics.json  
  このファイルには、分析対象のデータセット内の各フィーチャに対する列状の統計が含まれます  
- constraints.json  
  このファイルには、確認されたフィーチャの制約が含まれます。

In [ ]:
s3_client = boto3.Session().client("s3")
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get("Key") for report_file in result.get("Contents")]
print("Found Files:")
print("\n ".join(report_files))

### スケジュールの作成

In [ ]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

## 2. データの質に問題がないかどうか収集したデータを分析

上記のデータを収集したら、モニタリング・スケジュールでデータを分析・監視する

In [ ]:
constraints_df = pd.io.json.json_normalize(
    baseline_job.suggested_constraints().body_dict["features"]
)
constraints_df.head(10)

In [ ]:
# まず、いくつかのテストスクリプトをS3バケットにコピーして、前後の処理に使用できるようにします。
boto3.Session().resource("s3").Bucket(bucket).Object(code_prefix + "/preprocessor.py").upload_file(
    "preprocessor.py"
)
boto3.Session().resource("s3").Bucket(bucket).Object(code_prefix + "/postprocessor.py").upload_file(
    "postprocessor.py"
)

先に作成したエンドポイントのモデル監視スケジュールを作成することができます。ベースラインのリソース（制約条件や統計情報）を使用して、リアルタイムのトラフィックと比較します。

In [ ]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

mon_schedule_name = "DEMO-xgb-churn-pred-model-monitor-schedule-" + strftime(
    "%Y-%m-%d-%H-%M-%S", gmtime()
)
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint, # モニタリング対象のエンドポイント
    # record_preprocessor_script=pre_processor_script,
    post_analytics_processor_script=s3_code_postprocessor_uri, # 後処理するスクリプトをS3に事前に用意しておいて、そのパスを指定する。
    output_s3_uri=s3_report_path, # モニタリング結果を出力するS3パス
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

### 人工的なトラフィックの生成を開始

以下のセルは、エンドポイントにいくつかのトラフィックを送信するスレッドを開始します。このスレッドを終了させるには、カーネルを停止する必要があることに注意してください。トラフィックがない場合、処理するデータがないため、監視ジョブは `Failed` とマークされます。

In [ ]:
from threading import Thread
from time import sleep
import time

endpoint_name = predictor.endpoint
runtime_client = boto3.client("runtime.sagemaker")

# (just repeating code from above for convenience/ able to run this section independently)
def invoke_endpoint(ep_name, file_name, runtime_client):
    with open(file_name, "r") as f:
        for row in f:
            payload = row.rstrip("\n")
            response = runtime_client.invoke_endpoint(
                EndpointName=ep_name, ContentType="text/csv", Body=payload
            )
            response["Body"].read()
            time.sleep(1)


def invoke_endpoint_forever():
    while True:
        invoke_endpoint(endpoint_name, "test_data/test-dataset-input-cols.csv", runtime_client)


thread = Thread(target=invoke_endpoint_forever)
thread.start()

# Note that you need to stop the kernel to stop the invocations

### スケジュールの説明と点検

記述したら、MonitoringScheduleStatusがScheduledに変わることを確認してください。

In [ ]:
desc_schedule_result = my_default_monitor.describe_schedule()
print("Schedule status: {}".format(desc_schedule_result["MonitoringScheduleStatus"]))

### 実行内容の一覧
スケジュールでは、あらかじめ指定した間隔でジョブを開始します。ここでは、最新の5つの実行結果が表示されています。1時間ごとのスケジュールを作成した後にこれを起動する場合、実行内容が空欄になっていることがありますのでご注意ください。実行の開始を確認するには、（UTCの）時間の境界を越えるまで待たなければならないかもしれません。以下のコードは、待機するためのロジックです。

注：1時間ごとのスケジュールであっても、Amazon SageMakerは実行をスケジュールするために20分のバッファ期間を持っています。実行の開始時間は、1時間の境界から0分から20分の間になることがあります。これは、バックエンドでのロードバランシングのために行われます。

In [ ]:
mon_executions = my_default_monitor.list_executions()
print(
    "We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.\nWe will have to wait till we hit the hour..."
)

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(60)
    mon_executions = my_default_monitor.list_executions()

### 特定の実行（最新の実行）の検査
前のセルでは、最新の完了または失敗したスケジュール実行をピックアップしました。ここでは、考えられるターミナルの状態と、それぞれの意味を説明します。

* Completed - モニタリングの実行が完了し、違反レポートに問題が見つからなかったことを意味します。
* CompletedWithViolations - これは、実行が完了したが、制約違反が検出されたことを意味します。
* Failed - モニタリングの実行が失敗したことを意味します。原因は、クライアントのエラー（おそらく間違ったロールのプレミッション）またはインフラストラクチャの問題です。何が起こったかを正確に特定するには、FailureReasonとExitMessageのさらなる調査が必要です。
* Stopped - ジョブが最大ランタイムを超えたか、手動で停止されました。

In [ ]:
latest_execution = mon_executions[
    -1
]  # latest execution's index is -1, second to last is -2 and so on..
time.sleep(60)
latest_execution.wait(logs=False)

print("Latest execution status: {}".format(latest_execution.describe()["ProcessingJobStatus"]))
print("Latest execution result: {}".format(latest_execution.describe()["ExitMessage"]))

latest_job = latest_execution.describe()
if latest_job["ProcessingJobStatus"] != "Completed":
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )

In [ ]:
report_uri = latest_execution.output.destination
print("Report Uri: {}".format(report_uri))

### 生成されたレポートの一覧

In [ ]:
from urllib.parse import urlparse

s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip("/")
print("Report bucket: {}".format(report_bucket))
print("Report key: {}".format(report_key))

s3_client = boto3.Session().client("s3")
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get("Contents")]
print("Found Report Files:")
print("\n ".join(report_files))

### 違反レポート

ベースラインと比較して違反があった場合は、ここに記載されます。

In [ ]:
violations = my_default_monitor.latest_monitoring_constraint_violations()
pd.set_option("display.max_colwidth", -1)
constraints_df = pd.io.json.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

### その他のコマンド
また、監視スケジュールの開始や停止も可能です。

In [ ]:
# my_default_monitor.stop_monitoring_schedule()
# my_default_monitor.start_monitoring_schedule()

## リソースの削除

データの収集を継続するために、エンドポイントの実行を継続することができます。これ以上データを収集したり、このエンドポイントを使用したりする予定がない場合は、追加料金の発生を避けるために、エンドポイントを削除してください。なお、エンドポイントを削除しても、モデルの起動時に取得したデータは削除されません。そのデータは、あなた自身が削除するまで、Amazon S3に残ります。

しかしその前に、まずスケジュールを削除する必要があります。

In [ ]:
my_default_monitor.delete_monitoring_schedule()
time.sleep(60)  # actually wait for the deletion

In [ ]:
predictor.delete_endpoint()

In [ ]:
#predictor.delete_model()